<a href="https://colab.research.google.com/github/cwcheng0/2019_Pixnet_hackathon/blob/master/2019Pixnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## Read from Google
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_absolute_error as MAE
import gc
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML


In [2]:
# Mount personal Google Drive; all data files are in personal Google Drive
# account: My main account @ Gmail
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Use PDrive function
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 1.9MB/s 


In [4]:
# List files in root directory
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: 2019.8.27 北北海道地區FIT觀光說明會邀請函_台北場次, id: 1acsTFcZ6sR_RbI3Egz3UPequiO-hu6izF6gIfA_S6U8
title: PixnetData, id: 1zpLvHPbjbntAGzNfhYeQKo4I5PtIXmVw
title: Pixnet整理_Mike_20190804, id: 163PlI9NvQ5wVl62z6Crp4jihVWA4nnABRnPS8tU9neY
title: Tourism, id: 1rl2L8Vxmj4mqqFNCa23NUaypbVKazJRD
title: Python, id: 1dFGhOQOJ9t5x5bJUph24v_N-mnmBKsMZ
title: LionData, id: 192h03HmhseVUECi1YL9BAH44nzOAOl99
title: Colab Notebooks, id: 13yubYLLF_3_MObjXoj-K2aGo-_UFoQZo
title: bnbdata, id: 1yWUzpFHcZDftSVqbCaepJVd9oGsm2pww


In [5]:
# read data from an existing file to see if a command is correct?
f = open("/content/gdrive/My Drive/LionData/foo.txt", "r")
print(f.read())


Hello Google Drive!


In [6]:
# Read data
pd = pd.read_csv('/content/gdrive/My Drive/PixnetData/OriginalData/HotelsCombined API/Hackathon_Hotels_TW_EN.csv', header=None)

pd.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,ChainID,ChainName,CountryCode,CountryFileName,CountryName,CurrencyCode,Facilities,HotelAddress,HotelFileName,HotelID,HotelName,HotelPostcode,ImageID,Images,Latitude,Longitude,MinRate,NumberOfReviews,OverallRating,PlaceFileName,PlaceID,PlaceName,PlaceType,Popularity,PropertyType,PropertyTypeID,StarRating,StateName,StatePlacefilename,StatePlaceID,Themes,Trademarked
1,Null,NaN,TW,Taiwan,Taiwan,TWD,1|2|5|10|19|20|21|60|181|202|203|209|210|211,16 Kun Yuan Road,Friends_Yu_Pin_Hotel_Banciao_City,1652140,Friends Yu Pin Hotel Banciao City,NaN,Null,NaN,25.011741700000,121.460618600000,Null,5,6,New_Taipei,296823,New Taipei City,City,1875612,Hotel,0,2,NaN,NaN,Null,NaN,False
2,Null,NaN,TW,Taiwan,Taiwan,TWD,21|40|41|51|53|56|209|210|211,"3F, No.15, Ishin 2nd Road",Yes_Hotel_Kaohsiung,1652552,Yes Hotel Kaohsiung,NaN,Null,NaN,22.610372500000,120.312233000000,Null,0,Null,Kaohsiung,296791,Kaohsiung City,City,2163648,Hotel,0,2,NaN,NaN,Null,6,False
3,Null,NaN,TW,Taiwan,Taiwan,TWD,21|31|53|56|60,"No. 216, Dong-Wen Aw, Wenshan Rd.",Dolphin_Bay_Hotel_Makung,1652589,Dolphin Bay Hotel Makung,NaN,Null,NaN,23.559932500000,119.583457700000,Null,0,Null,Magong_City,48813,Magong City,Town,2315015,Hotel,0,4,NaN,NaN,Null,NaN,False
4,Null,NaN,TW,Taiwan,Taiwan,TWD,5|17|19|21|39|51|53|60|66|139|202|209|210|211,"No. 10, Ln. 291, Sec. 2, Daren Road, Wuci Dist...",A_Boutique_Hotel,1652613,A+ Boutique Hotel,NaN,Null,NaN,24.262567500000,120.536354100000,Null,0,Null,Taichung,296793,Taichung City,City,1901192,Hotel,0,3,NaN,NaN,Null,NaN,False


In [7]:
# Check first 5 rows in the targeted file.
import gzip
import pandas as pd

df5 = pd.read_json('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000005.gz', compression='gzip', lines=True)
  
#df2 = pd.read_csv('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000005.gz', compression='gzip', header=0, sep=' ', quotechar='"', error_bad_lines=False)                  
#df2 = pd.read_csv('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000005.gz', compression='gzip', header=0, sep=',', quotechar='"')

df5.head()

,author_id,browser,browser_version,category_id,city,cookie_age,cookie_gender,cookie_hash,country,date,device_brand,device_marketing,device_model,hour,os,os_version,referrer_host,referrer_url,referrer_venue,resolution,timestamp,url
0,wiki0918,Chrome,73.0.3683,財經企管,,不明,不明,rkLnxhYULJKA1N1rHRy0hQ==,TW,20190403,,,Other,23,Windows,,wiki0918.pixnet.net,http://wiki0918.pixnet.net/blog/post/164541912...,pixnet,1120x700,2019-04-03 15:18:22,http://wiki0918.pixnet.net/blog/post/164541912
1,ofeyhong,Chrome,75.0.3770,數位生活,Bade,不明,男,LbJV3CsF/57SsDbjuz7bhg==,TW,20190701,,,Other,23,Windows,,ofeyhong.pixnet.net,https://ofeyhong.pixnet.net/blog,pixnet,2560x1080,2019-07-01 15:05:08,https://ofeyhong.pixnet.net/blog/post/223618134
2,ryanyang0725,Chrome,73.0.3683,國內旅遊,,不明,不明,j+ftuAcka7hKI0DSwG9y4A==,TW,20190403,,,Other,14,Windows 7,,ryanyang0725.pixnet.net,http://ryanyang0725.pixnet.net/blog/post/21216...,pixnet,1093x615,2019-04-03 06:42:01,http://ryanyang0725.pixnet.net/blog/post/21216...
3,JessicaYiNingWu,Mobile Safari,12.0,國外旅遊,,不明,女,9hw6RDjGYiW+WxcfeCY36A==,TW,20190303,Apple,iPad,iPad,22,iOS,12.1.4,jessicayiningwu.pixnet.net,http://jessicayiningwu.pixnet.net/blog/post/22...,pixnet,834x1112,2019-03-03 14:28:07,http://jessicayiningwu.pixnet.net/blog/post/22...
4,queen010x,Chrome,73.0.3683,不設分類,Taipei,35-45,男,6u6MJHyoFoz5RTIThJaP7w==,TW,20190331,,,Other,11,Windows,,queen010x.pixnet.net,http://queen010x.pixnet.net/blog/post/27924126...,pixnet,1344x840,2019-03-31 03:18:19,http://queen010x.pixnet.net/blog/post/279241265


In [9]:
# Check first 5 rows in the targeted file.
df4 = pd.read_json('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000004.gz', compression='gzip', lines=True)

df4.head()

,author_id,browser,browser_version,category_id,city,cookie_age,cookie_gender,cookie_hash,country,date,device_brand,device_marketing,device_model,hour,os,os_version,referrer_host,referrer_url,referrer_venue,resolution,timestamp,url
0,old0127,Chrome,72.0.3626,生活綜合,,不明,女,ROtnfGdQouafeKtts1UffA==,TW,20190304,,,Other,6,Windows 7,,old0127.pixnet.net,http://old0127.pixnet.net/blog/post/323056995-...,pixnet,1344x840,2019-03-03 22:01:49,http://old0127.pixnet.net/blog/post/323056995
1,iface,Firefox Mobile,65.0,醫療保健,Taipei,25-35,女,vZtBcI1TPyIMypwEYwN0rA==,TW,20190324,,,Generic Smartphone,23,Android,,www.google.com,https://www.google.com/search?q=%E6%8C%87%E7%9...,google,324x527,2019-03-24 15:09:20,https://iface.pixnet.net/blog/post/102459892
2,ofeyhong,Chrome Mobile,71.0.3578,數位生活,Taipei,不明,男,3AnwReSJfbgmGZUmps09Iw==,TW,20190211,,,Generic Smartphone,1,Android,8.0.0,www.googleapis.com,https://www.googleapis.com/auth/chrome-content...,google,424x848,2019-02-10 17:32:26,https://ofeyhong.pixnet.net/blog/post/223090374
3,cwlee,IE,8.0,活動紀錄,Debrecen,不明,不明,rDq2yFvKZb/fbFiGCvxPqA==,HU,20181123,,,Other,8,Windows XP,,cwlee.pixnet.net,http://cwlee.pixnet.net/blog,pixnet,945x723,2018-11-23 00:54:22,http://cwlee.pixnet.net/blog/post/67011141
4,cocowuno1,Chrome Mobile,66.0.3359,數位生活,Taipei,18-25,女,VRoYMDU+WwQ2fsFHRCPuiA==,TW,20190204,,,SM-J600G,5,Android,8.0.0,com.google.android.googlequicksearchbox,android-app://com.google.android.googlequickse...,google,320x658,2019-02-03 21:24:01,https://cocowuno1.pixnet.net/blog/post/463162508


In [10]:
# Check first 5 rows in the targeted file.
df3 = pd.read_json('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000003.gz', compression='gzip', lines=True)

df3.head()

,author_id,browser,browser_version,category_id,city,cookie_age,cookie_gender,cookie_hash,country,date,device_brand,device_marketing,device_model,hour,os,os_version,referrer_host,referrer_url,referrer_venue,resolution,timestamp,url
0,yoyoman822,Other,,國內旅遊,,不明,不明,AP8b905qb4NyfN2KKFX5cg==,TW,20190227,,,Other,9,Windows 7,,yoyoman822.pixnet.net,http://yoyoman822.pixnet.net/blog/post/6429749...,pixnet,960x540,2019-02-27 01:00:58,http://yoyoman822.pixnet.net/blog/post/66396600
1,ofeyhong,Chrome,74.0.3729,數位生活,,25-35,男,ozuzoWp5RZ1TsQ6bXUxElQ==,TW,20190528,,,Other,14,Windows,,ofeyhong.pixnet.net,https://ofeyhong.pixnet.net/blog/post/21767679...,pixnet,1920x1200,2019-05-28 06:40:41,https://ofeyhong.pixnet.net/blog/post/217676790
2,a122557788,Chrome,73.0.3683,數位生活,,不明,不明,g7c6Bd8e3Fo//kGT2VPKZw==,TW,20190422,,,Other,9,Windows 7,,a122557788.pixnet.net,http://a122557788.pixnet.net/blog/post/2727455...,pixnet,1093x615,2019-04-22 01:01:12,http://a122557788.pixnet.net/blog/post/272745560
3,aqua02182005,Chrome,75.0.3770,美味食記,,25-35,女,vufO1VPTeoeLBqoo5nEukw==,TW,20190627,,,Other,12,Windows 7,,aqua02182005.pixnet.net,https://aqua02182005.pixnet.net/blog/post/3502...,pixnet,1280x768,2019-06-27 04:14:52,https://aqua02182005.pixnet.net/blog/post/3502...
4,daintyshoopy,IE,10.0,美容彩妝,Taichung,不明,不明,V8WIkw5iCq656kC2A/wmGA==,TW,20190324,,,Other,4,Windows 8,,daintyshoopy.pixnet.net,http://daintyshoopy.pixnet.net/blog/post/46706...,pixnet,1360x768,2019-03-23 20:11:03,http://daintyshoopy.pixnet.net/blog/post/46712...


In [11]:
# Check first 5 rows in the targeted file.
df2 = pd.read_json('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000002.gz', compression='gzip', lines=True)

df2.head()

,author_id,browser,browser_version,category_id,city,cookie_age,cookie_gender,cookie_hash,country,date,device_brand,device_marketing,device_model,hour,os,os_version,referrer_host,referrer_url,referrer_venue,resolution,timestamp,url
0,simgag,Firefox,66.0,美味食記,Taipei,25-35,女,HljaCCkXzysoJZ39dY71iA==,TW,20190511,,,Other,14,Windows,,r.search.yahoo.com,https://r.search.yahoo.com/_ylt=AwrtahITaNZc5l...,yahoo,1504x1003,2019-05-11 06:18:32,http://simgag.pixnet.net/blog/post/343622240
1,s8dr,Chrome,74.0.3729,生活綜合,Taipei,35-45,男,PuJMfJTD9bDOWBzC5nJ5ww==,TW,20190528,,,Other,0,Windows,,s8dr.pixnet.net,http://s8dr.pixnet.net/blog/post/63198631-%E3%...,pixnet,2560x1440,2019-05-27 16:44:59,http://s8dr.pixnet.net/blog/post/63198631
2,chefcch,Mobile Safari,12.0,不設分類,Taipei,不明,不明,lOCLo48krLy4N2EnQX9vBA==,TW,20190125,Apple,iPad,iPad,16,iOS,12.1.1,chefcch.pixnet.net,http://chefcch.pixnet.net/blog/post/400822052,pixnet,834x1112,2019-01-25 08:11:05,http://chefcch.pixnet.net/blog/post/400822052
3,era10156,Chrome,69.0.3497,美味食記,Kaohsiung,不明,不明,bb2mlX5bw/qwanvHc1C3PA==,TW,20180923,,,Other,14,Windows,,l.facebook.com,https://l.facebook.com/,facebook,2560x1080,2018-09-23 06:51:34,http://era10156.pixnet.net/blog/post/32441302
4,cott6226,Android,4.2.2,圖文創作,Taipei,18-25,女,WDt6Dcu+QgG+0tSfLNecXg==,TW,20181027,,,2013023,15,Android,4.2.2,www.google.com.tw,http://www.google.com.tw/search?q=%E6%9E%97%E5...,google,720x1280,2018-10-27 07:52:24,https://cott6226.pixnet.net/blog/post/325915701


In [12]:
# Check first 5 rows in the targeted file.
df1 = pd.read_json('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000001.gz', compression='gzip', lines=True)

df1.head()

,author_id,browser,browser_version,category_id,city,cookie_age,cookie_gender,cookie_hash,country,date,device_brand,device_marketing,device_model,hour,os,os_version,referrer_host,referrer_url,referrer_venue,resolution,timestamp,url
0,cash2019,IE,10.0,職場甘苦,,不明,不明,Z0UCO61oUx+mHSZjtB2U2Q==,FR,20190621,,,Other,2,Windows 7,,cash2019.pixnet.net,http://cash2019.pixnet.net/blog/post/278638668,pixnet,1364x768,2019-06-20 18:53:32,http://cash2019.pixnet.net/blog/post/278641992
1,yehman,IE,8.0,美味食記,Nanning,不明,不明,z1cOWHqFdjAVJ4KHztue+A==,CN,20181027,,,Other,2,Windows XP,,yehman.pixnet.net,http://yehman.pixnet.net/blog/post/47976603,pixnet,2560x1440,2018-10-26 18:54:10,http://yehman.pixnet.net/blog/post/48504321
2,vici1717,Chrome,63.0.3239,生活綜合,Mosul,不明,不明,k6+TECYRSdSub+TuY14ing==,IQ,20181024,,,Other,19,Windows 7,,vici1717.pixnet.net,http://vici1717.pixnet.net/blog,pixnet,1152x864,2018-10-24 11:18:39,http://vici1717.pixnet.net/blog/post/67000521
3,neubrise29,Chrome,56.0.2924,小說連載,Taipei,18-25,女,/CzPiXkc09wxejvKKKUxUQ==,TW,20181231,Samsung,,SAMSUNG SM-T385,23,Android,7.1.1,neubrise29.pixnet.net,http://neubrise29.pixnet.net/blog/post/110347916,pixnet,962x601,2018-12-31 15:58:42,http://neubrise29.pixnet.net/blog/post/110347952
4,pro770906520,Firefox,3.0.8,國內旅遊,,不明,不明,YKu8n8q3Xq9u4nO2AaDLUw==,,20190331,,,Other,7,Windows XP,,www.google.com.tw,http://www.google.com.tw,google,1716x994,2019-03-30 23:12:54,http://pro770906520.pixnet.net/blog/post/85753902


In [13]:
# Check first 5 rows in the targeted file.
df0 = pd.read_json('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000000.gz', compression='gzip', lines=True)

df0.head()

,author_id,browser,browser_version,category_id,city,cookie_age,cookie_gender,cookie_hash,country,date,device_brand,device_marketing,device_model,hour,os,os_version,referrer_host,referrer_url,referrer_venue,resolution,timestamp,url
0,jpsex,IE,7.0,醫療保健,Andong,不明,不明,6UUJi+mcLErwqrk8DAMnOA==,TW,20190403,,,Other,19,Windows,,r.search.yahoo.com,https://r.search.yahoo.com/_ylt=AwrtSCJvmKRcKk...,yahoo,1219x914,2019-04-03 11:27:18,http://jpsex.pixnet.net/blog/post/4782770
1,jcyufen850509,Chrome,73.0.3683,不設分類,Taipei,45-65,女,/IEsDWMiD2HZPJZ/UWOxlQ==,TW,20190414,,,Other,8,Windows,,jcyufen850509.pixnet.net,http://jcyufen850509.pixnet.net/blog/post/5204...,pixnet,1120x700,2019-04-14 00:12:35,http://jcyufen850509.pixnet.net/blog/post/5204...
2,haruhii,Chrome,75.0.3770,美味食記,,不明,女,q0Ec094T71lafn6R6R4H2Q==,TW,20190627,,,Other,14,Windows 7,,haruhii.pixnet.net,https://haruhii.pixnet.net/blog/post/31341951-...,pixnet,1143x715,2019-06-27 06:24:53,https://haruhii.pixnet.net/blog/post/43466162
3,cott6226,Firefox Mobile,65.0,圖文創作,Taipei,不明,男,kxpD4fbuvu2xrfeYvdt7Iw==,TW,20190602,,,Generic Smartphone,1,Android,8.0.0,www.google.com,https://www.google.com/search?safe=off&client=...,google,312x573,2019-06-01 17:04:18,https://cott6226.pixnet.net/blog/post/349391359
4,rich0178,IE,10.0,小說連載,,不明,不明,jYhgEaPAuBTxQ3DELsDTBg==,TW,20181222,,,Other,20,Windows 7,,rich0178.pixnet.net,http://rich0178.pixnet.net/blog/post/104155642,pixnet,1400x1050,2018-12-22 12:41:48,http://rich0178.pixnet.net/blog/post/83932074


In [0]:
# Check first 5 rows in the targeted file.
df_ar = pd.read_json('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_url_articles.gz', compression='gzip', lines=True)

df_ar.head()